In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [5]:
#Load data
fies_data = pd.read_csv('FIES PUF 2023 Volume1.CSV')

#Check columns 
fies_data.head()

,W_REGN,W_PROV,SEQ_NO,RPROV,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,...,RFACT,MEM_RFACT,URB,PERCAPITA,NPCINC,RPCINC,PRPCINC,PPCINC,RPCINC_NIR,W_REGN_NIR
0,1,28,1,2800,2.5,119000,0,119000,0,461000,...,88.543663,229.392223,2,242828.000000,10,10,10,10,10,1
1,1,28,2,2800,6.0,154400,0,154400,0,142000,...,88.543663,550.541336,2,68663.335938,5,5,2,2,5,1
2,1,28,3,2800,3.5,683452,0,683452,0,37000,...,88.543663,321.149113,2,233774.859375,10,10,10,10,10,1
3,1,28,4,2800,2.5,48200,0,48200,10000,55000,...,88.543663,229.392223,2,103415.203125,7,8,5,5,8,1
4,1,28,5,2800,3.0,400994,0,400994,0,108000,...,88.543663,275.270668,2,183574.671875,9,10,9,9,10,1


In [6]:
#Check columns
print(fies_data.columns)

Index(['W_REGN', 'W_PROV', 'SEQ_NO', 'RPROV', 'FSIZE', 'REG_SAL', 'SEASON_SAL',
       'WAGES', 'NETSHARE', 'CASH_ABROAD', 'CASH_DOMESTIC', 'RENTALS_REC',
       'INTEREST', 'PENSION', 'DIVIDENDS', 'OTHER_SOURCE', 'NET_RECEIPT',
       'REGFT', 'NET_CFG', 'NET_LPR', 'NET_FISH', 'NET_FOR', 'NET_RET',
       'NET_MFG', 'NET_TRANS', 'NET_NEC_A8', 'NET_NEC_A9', 'NET_NEC_A10',
       'EAINC', 'LOSSES', 'BREAD', 'MEAT', 'FISH', 'MILK', 'OIL', 'FRUIT',
       'VEG', 'SUGAR', 'FOOD_NEC', 'FRUIT_VEG', 'COFFEE', 'TEA', 'COCOA',
       'WATER', 'SOFTDRINKS', 'OTHER_NON_ALCOHOL', 'ALCOHOL', 'TOBACCO',
       'OTHER_VEG', 'SERVICES_PRIMARY_GOODS', 'ALCOHOL_PROCDUCTION_SERVICES',
       'FOOD_HOME', 'FOOD_OUTSIDE', 'FOOD', 'CLOTH', 'HOUSING_WATER',
       'ACTRENT', 'IMPUTED_RENT', 'BIMPUTED_RENT', 'RENTVAL', 'FURNISHING',
       'HEALTH', 'TRANSPORT', 'COMMUNICATION', 'RECREATION', 'EDUCATION',
       'INSURANCE', 'MISCELLANEOUS', 'DURABLE', 'OCCASION',
       'OTHER_EXPENDITURE', 'OTHER_DISBURSEME

In [7]:
# Ensure numeric conversion for key columns
expenditure_cols = ['TOTEX', 'FOOD', 'HOUSING_WATER', 'TRANSPORT', 'TOINC', 'FSIZE']
fies_data[expenditure_cols] = fies_data[expenditure_cols].apply(pd.to_numeric, errors='coerce')

# Compute per capita total expenditure
fies_data['TOTEX_PC'] = fies_data['TOTEX'] / fies_data['FSIZE']

# Group by region code and compute averages
region_stats = fies_data.groupby('W_REGN').agg({
    'TOTEX_PC': 'mean',
    'FOOD': 'mean',
    'HOUSING_WATER': 'mean',
    'TRANSPORT': 'mean',
    'TOINC': 'mean'
}).reset_index()

region_stats.columns = ['Region', 'Avg_TOTEX_PC', 'Avg_FOOD', 'Avg_HOUSING', 'Avg_TRANSPORT', 'Avg_TOINC']

In [8]:
print(region_stats.sort_values(by='Avg_TOTEX_PC', ascending=False).head(10))  # Top 10 regions by expenditure
print(region_stats.sort_values(by='Avg_TOTEX_PC').head(10))  # Lowest 10 regions by expenditure


    Region   Avg_TOTEX_PC       Avg_FOOD    Avg_HOUSING  Avg_TRANSPORT  \
12      13  111143.823328  140536.759006  108601.355679   23057.931416   
3        4   86766.174714  120197.844329   69337.691638   21675.673737   
2        3   84476.433746  117807.528730   67327.239868   19197.906790   
0        1   72839.676294  106945.637026   51078.168227   17093.407426   
13      14   71674.861989   99049.375212   51052.801862   14852.152150   
5        6   69090.564771  100481.643284   46890.306038   15953.422821   
6        7   65840.122344   92849.467752   55361.075653   16270.371681   
1        2   65148.460676   89243.253090   46353.186468   14881.528542   
15      17   59574.410972   85493.056503   39381.707070   12947.671446   
9       10   59317.474974   87987.218153   45422.467184   15716.405012   

        Avg_TOINC  
12  502607.281727  
3   401832.399452  
2   369237.469217  
0   372430.551630  
13  346274.761274  
5   317364.960585  
6   342940.960393  
1   321230.716920  
15  2

In [9]:
#Top 5 highest and lowest expenditure regions
top_regions = region_stats.nlargest(5, 'Avg_TOTEX_PC')['Region'].tolist()
bottom_regions = region_stats.nsmallest(5, 'Avg_TOTEX_PC')['Region'].tolist()

print("Top expenditure regions:", top_regions)
print("Lowest expenditure regions:", bottom_regions)


Top expenditure regions: [13, 4, 3, 1, 14]
Lowest expenditure regions: [19, 9, 11, 5, 12]


In [10]:
region_name_map = {
    1: "Region I - Ilocos Region",
    2: "Region II - Cagayan Valley",
    3: "Region III - Central Luzon",
    4: "Region IVA - CALABARZON",
    5: "Region V- Bicol",
    6: "Region VI - Western Visayas",
    7: "Region VII - Central Visayas",
    8: "Region VIII - Eastern Visayas",
    9: "Region IX - Zamboanga Peninsula",
    10: "Region X - Northern Mindanao",
    11: "Region XI - Davao",
    12: "Region XII - SOCCSKSARGEN",
    13: "National Capital Region",
    14: "Cordillera Administrative Region",
    16: "Region XIII - Caraga",
    17: "Region IVB - MIMAROPA",
    19: "Bangsamoro Autonomous Region in Muslim Mindanao"
}


In [16]:
top_values = region_stats.set_index('Region').loc[top_regions, 'Avg_TOTEX_PC'].values
bottom_values = region_stats.set_index('Region').loc[bottom_regions, 'Avg_TOTEX_PC'].values

top_df = pd.DataFrame({
    "Rank": range(1, 6),
    "Region": top_regions_named,
    "Avg_TOTEX_PC": top_values
})

bottom_df = pd.DataFrame({
    "Rank": range(1, 6),
    "Region": bottom_regions_named,
    "Avg_TOTEX_PC": bottom_values
})

In [17]:
print("Top 5 Regions by Per Capita Expenditure")
print(top_df.to_string(index=False))

print("\nBottom 5 Regions by Per Capita Expenditure")
print(bottom_df.to_string(index=False))

Top 5 Regions by Per Capita Expenditure
 Rank                           Region  Avg_TOTEX_PC
    1          National Capital Region 111143.823328
    2          Region IVA - CALABARZON  86766.174714
    3       Region III - Central Luzon  84476.433746
    4         Region I - Ilocos Region  72839.676294
    5 Cordillera Administrative Region  71674.861989

Bottom 5 Regions by Per Capita Expenditure
 Rank                                          Region  Avg_TOTEX_PC
    1 Bangsamoro Autonomous Region in Muslim Mindanao  40208.640249
    2                 Region IX - Zamboanga Peninsula  53359.597901
    3                               Region XI - Davao  53362.785730
    4                                 Region V- Bicol  54569.444147
    5                       Region XII - SOCCSKSARGEN  56286.176256
